In [ ]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer
import numpy as np

# Constants
MAX_LEN = 128
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Student Model (LSTM)
class StudentModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes, tokenizer):
        super(StudentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=tokenizer.pad_token_id)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(0.25)
        self.classifier = nn.Linear(hidden_dim * 2, num_classes)
        self.match_hidden = nn.Linear(hidden_dim * 2, 768)  # Match với BERT

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        # Use mean pooling of all hidden states
        last_hidden = torch.mean(lstm_out, dim=1)  # Take mean across sequence length dimension
        last_hidden = self.dropout(last_hidden)
        matched_hidden = self.match_hidden(last_hidden)  # Đưa về 768 chiều
        logits = self.classifier(last_hidden)
        return logits, matched_hidden

def load_model(model_path):
    # Khởi tạo tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Khởi tạo model
    model = StudentModel(
        vocab_size=tokenizer.vocab_size,
        embedding_dim=256,
        hidden_dim=256,
        num_classes=4,
        tokenizer=tokenizer  # Thêm tokenizer vào đây
    ).to(DEVICE)

    # Load weights
    checkpoint = torch.load(model_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    return model, tokenizer

def predict_text(model, tokenizer, text):
    # Tokenize text
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=False,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(DEVICE)

    # Predict
    with torch.no_grad():
        logits, _ = model(input_ids)
        probs = torch.softmax(logits, dim=1)

    # Lấy xác suất cho từng class
    probs = probs.cpu().numpy()[0]

    # In kết quả
    class_names = ['World', 'Sports', 'Business', 'Sci/Tech']
    print("\nPrediction Results:")
    for i, (class_name, prob) in enumerate(zip(class_names, probs)):
        print(f"{class_name}: {prob*100:.2f}%")

# Load model
model_path = '/content/drive/MyDrive/ML_DM/Student model/acc 87/best_student_model.pth'
model, tokenizer = load_model(model_path)
print("Model loaded successfully!")

# Test với một đoạn text
text = "Apple announces new iPhone with advanced AI features"
predict_text(model, tokenizer, text)

Model loaded successfully!

Prediction Results:
World: 2.00%
Sports: 0.03%
Business: 1.35%
Sci/Tech: 96.62%


In [ ]:
import ipywidgets as widgets
from IPython.display import display

text_input = widgets.Text(
    description='Text:',
    layout=widgets.Layout(width='500px')  # hoặc '100%' cho full width
)
button = widgets.Button(description='Predict')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        predict_text(model, tokenizer, text_input.value)

button.on_click(on_button_clicked)
display(text_input, button, output)

Text(value='', description='Text:', layout=Layout(width='500px'))

Button(description='Predict', style=ButtonStyle())

Output()